<h2>Capture Images</h2>


In [8]:
import cv2
import os


def capture_image(student_name, num_images=50):
    # Create folder for the student if it doesn't exist
    if not os.path.exists(f"dataset/{student_name}"):
        os.makedirs(f"dataset/{student_name}")

    # Initialize webcam
    cap = cv2.VideoCapture(0)
    cv2.namedWindow("Capture", cv2.WINDOW_NORMAL)

    print(
        f"Press 'C' to capture {num_images} images for {student_name} or 'Q' to quit..."
    )

    count = 0
    while count < num_images:
        ret, frame = cap.read()  # Read frame from the webcam
        if ret:
            cv2.imshow("Capture", frame)  # Show live feed

            key = cv2.waitKey(1) & 0xFF  # Capture key press

            if key == ord("c"):  # Press 'C' to capture image
                image_path = f"dataset/{student_name}/image{count+1}.jpg"
                cv2.imwrite(image_path, frame)  # Save the image
                print(f"Captured image {count+1} for {student_name}: {image_path}")
                count += 1

            elif key == ord("q"):  # Press 'Q' to quit capturing
                print("Quitting image capture.")
                break

    # Release webcam and destroy windows
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)


# Add new student names in the array
studentNames = ["Abdullah Nawaz"]
for studentName in studentNames:
    capture_image(studentName, num_images=5)

Press 'C' to capture 5 images for Abdullah Nawaz or 'Q' to quit...
Captured image 1 for Abdullah Nawaz: dataset/Abdullah Nawaz/image1.jpg
Captured image 2 for Abdullah Nawaz: dataset/Abdullah Nawaz/image2.jpg
Captured image 3 for Abdullah Nawaz: dataset/Abdullah Nawaz/image3.jpg
Captured image 4 for Abdullah Nawaz: dataset/Abdullah Nawaz/image4.jpg
Captured image 5 for Abdullah Nawaz: dataset/Abdullah Nawaz/image5.jpg


<h1>Dataset Processing</h1>


In [2]:
import os
import random
import numpy as np
import cv2
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf


# Preprocess function
def preprocess_image(image_path, image_size=(128, 128)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, image_size)
    img = img / 255.0
    img = np.expand_dims(img, axis=-1)
    return img


# Create image pairs function
def create_image_pairs(image_dir):
    students = os.listdir(image_dir)
    pairs = []
    labels = []

    # Generate positive pairs (same student)
    for student in students:
        student_path = os.path.join(image_dir, student)
        if not os.path.isdir(student_path):
            continue
        student_images = os.listdir(student_path)
        for i in range(len(student_images)):
            for j in range(i + 1, len(student_images)):
                img1 = f"{image_dir}/{student}/{student_images[i]}"
                img2 = f"{image_dir}/{student}/{student_images[j]}"
                pairs.append([preprocess_image(img1), preprocess_image(img2)])
                labels.append(1)

    # Generate negative pairs (different students)
    for i in range(len(students)):
        for j in range(i + 1, len(students)):
            student1_path = os.path.join(image_dir, students[i])
            student2_path = os.path.join(image_dir, students[j])
            if not os.path.isdir(student1_path) or not os.path.isdir(student2_path):
                continue
            student1_images = os.listdir(student1_path)
            student2_images = os.listdir(student2_path)
            img1 = f"{image_dir}/{students[i]}/{random.choice(student1_images)}"
            img2 = f"{image_dir}/{students[j]}/{random.choice(student2_images)}"
            pairs.append([preprocess_image(img1), preprocess_image(img2)])
            labels.append(0)

    return np.array(pairs), np.array(labels)


image_dir = "dataset"
pairs, labels = create_image_pairs(image_dir)

<h1>Train Model</h1>


In [3]:
# Convert to tensors
pairs = tf.convert_to_tensor(pairs, dtype=tf.float32)
labels = tf.convert_to_tensor(labels, dtype=tf.float32)


# Define base network
def create_base_network(input_shape=(128, 128, 1)):
    input_layer = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (7, 7), activation="relu", padding="same")(input_layer)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (5, 5), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation="relu")(x)
    return Model(input_layer, x)


# Create Siamese network
def create_siamese_network(input_shape=(128, 128, 1)):
    base_network = create_base_network(input_shape)
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    distance = layers.Lambda(lambda tensors: tf.math.abs(tensors[0] - tensors[1]))(
        [processed_a, processed_b]
    )
    output = layers.Dense(1, activation="sigmoid")(distance)
    return Model([input_a, input_b], output)


# Instantiate and compile the model
siamese_model = create_siamese_network()
siamese_model.compile(
    optimizer=Adam(0.0001), loss=BinaryCrossentropy(), metrics=["accuracy"]
)

# Train the model
history = siamese_model.fit(
    [pairs[:, 0], pairs[:, 1]], labels, batch_size=16, epochs=20, validation_split=0.2
)

# Save the model
siamese_model.save("model/siamese_model.keras")

Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_11', 'keras_tensor_12']. Received: the structure of inputs=('*', '*')
  warnings.warn(


  4/123 ━━━━━━━━━━━━━━━━━━━━ 1:56 977ms/step - accuracy: 0.9023 - loss: 0.6216

KeyboardInterrupt: 